In [1]:
from langchain.prompts import PromptTemplate
from src.commercial.templates import SYS_PALM_TEMPLATE, INST_PALM_TEMPLATE
from src.commercial.inference_palm import palm_completion
from utils.utils import get_clues
import pandas as pd

ModuleNotFoundError: No module named 'src'

In [2]:
sys_prompt_template = PromptTemplate.from_template(SYS_PALM_TEMPLATE)
inst_prompt_template = PromptTemplate.from_template(INST_PALM_TEMPLATE)
df = pd.read_csv('/home/t-sahuja/cultural_artifacts/clues/punjab/punjab_clues.csv')

In [3]:
clues = df['clues'].iloc[0].strip().split('\n')

In [6]:
sys_prompt = sys_prompt_template.format(cluelist=output)
inst_prompt = inst_prompt_template.format(state='Punjab')

In [7]:
inst_prompt

"\n   Name the object on the basis of the above clues from Punjab. After your guess, I will let you know if you are correct or not. If you're correct, do not make any further guesses and end the conversation. If you are wrong, make your second and final guess. I do not need to know your reasoning behind the answer. Just tell me the answer and nothing else. If you do not know the answer, say that you do not know the answer. Format your answer in the form of ANSWER: your_answer_here.\n"

In [16]:
palm_resp = palm_completion(sys_prompt = sys_prompt, inst_prompt=inst_prompt)

In [17]:
palm_resp.last

'ANSWER: Lassi\n\nIs this correct?'

In [18]:
palm_resp.reply('yes, this is correct').last

'I am glad that I was able to guess the correct answer. Lassi is a popular Punjabi drink made from curd. It is a refreshing and healthy drink that is often served during the summer. It is also a good source of protein and calcium.'

In [ ]:
df_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])

In [ ]:
def get_outputs(df, df_eval):
    for i, row in df.iterrows():
        print(f"artifact--{i}---")
        clues = row["clues"].strip().split("\n")
        artifact = row["artifact"].lower().strip()
        output = ""
        for j, clue in enumerate(clues):
            output += f"CLUE-{j+1}: {clue}\n"
            fin_clues = output.strip()
        
        palm_resp = palm_completion(sys_prompt=sys_prompt, inst_prompt=inst_prompt)
        palm_reply = palm_resp.last
        guess1 = palm_reply.split('\n')[0].split(':')[1].lower()
        if artifact in guess1:
            df_eval = df_eval.append(
                {
                    "guess1": guess1,
                    "guess2": "NA",
                    "ground_truth": artifact,
                    "clues": row["clues"],
                },
                ignore_index=True,
            )
            continue
        else:
            palm_2nd_resp = palm_resp.reply('your guess was not correct')
            palm_2nd_reply = palm_2nd_reply.last
            guess2 = palm_2nd_reply.split('\n')[0].split(':')[1].lower()
            if artifact in guess2:
                df_eval = df_eval.append(
                {
                    "guess1": guess1,
                    "guess2": guess2,
                    "ground_truth": artifact,
                    "clues": row["clues"],
                },
                ignore_index=True,
            )
            
            